# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
import findspark
findspark.init('/home/mint/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('project').getOrCreate()
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [3]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [15]:
from pyspark.ml.feature import StringIndexer
indexed = StringIndexer(inputCol = 'Cruise_line', outputCol = 'cruiseLine').fit(data).transform(data)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruiseLine|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|       1.0|
|    Elation|   Carnival| 15

In [17]:
assembler = VectorAssembler(inputCols = ['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density',
                                         'cruiseLine'],
                            outputCol = 'features')
output = assembler.transform(indexed)
# output.head(1)
final_data = output.select(['features', 'crew'])
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [18]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'crew')
model = lr.fit(train_data)
results = model.evaluate(test_data)
print(results.r2)
print(results.rootMeanSquaredError)

0.942201300150821
0.796048280707979


In [19]:
unlabeled_data = test_data.select('features')
predictions = model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,115.0,35.74,...|12.042258412898459|
|[6.0,90.0,20.0,9....| 10.29585592181097|
|[7.0,89.6,25.5,9....|11.225096278541551|
|[7.0,116.0,31.0,9...|12.698080231577627|
|[8.0,110.0,29.74,...|12.167730070600392|
|[9.0,81.0,21.44,9...| 9.603674957431105|
|[9.0,113.0,26.74,...|11.313560804884071|
|[9.0,116.0,26.0,9...|11.099509321831963|
|[10.0,91.62700000...|  9.17834689188943|
|[10.0,105.0,27.2,...|11.280506442404041|
|[10.0,110.0,29.74...|12.135386060167374|
|[11.0,90.09,25.01...| 8.883367313858114|
|[11.0,108.977,26....|11.052188872868369|
|[12.0,50.0,7.0,7....| 4.524915453490776|
|[12.0,88.5,21.24,...| 9.478202285715327|
|[12.0,91.0,20.32,...| 9.190881611068576|
|[13.0,25.0,3.82,5...|3.0403542092096605|
|[13.0,61.0,13.8,7...| 6.489873691398939|
|[13.0,63.0,14.4,7...| 6.681510550132149|
|[13.0,76.0,18.74,...|  8.85501239155996|
+--------------------+------------

In [22]:
from pyspark.sql.functions import corr
data.select(corr(data['crew'], data['cabins'])).show()
data.select(corr(data['crew'], data['passengers'])).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

